In [1]:
import pandas as pd
import numpy as np
import json
import os
from tqdm import tqdm
import requests
import time
import pickle

In [2]:
metadataDictionary = {}

def metadata(cik):
    '''
    Given a cik, it retrieves a dataframe with the metadata of that security for each 10Q and 10K form.
    Parameters:
        CIK: sec unique and non-reusable SEC identifier
    Global variables:
        metadataDictionary: if the metadata for a given CIK has been retrieved before, it will just return it instead of getting
        it again
    Returns:
        DataFrame with metadata
    '''
    cik = str(cik)
    if len(cik) != 10:
        cik = cik.zfill(10)
    if cik not in metadataDictionary:
        path = 'https://data.sec.gov/submissions/CIK{}.json'.format(cik)
        user_agent = 'jdjp_97@hotmail.com'
        headers = {'User-Agent':user_agent} 

        metadata = requests.get(path, headers=headers).json()
        filings = metadata['filings']
        recent = pd.DataFrame( filings['recent'] )
        recent = recent[recent['form'].isin(['10-Q', '10-K'])]
        files = filings['files']

        dataFrames = [recent]
        #Loop thourgh files
        for idx, i in enumerate(files):
            fileName = i['name']
            path2 = 'https://data.sec.gov/submissions/{}'.format(fileName)
            recent2 = pd.DataFrame(requests.get(path2, headers = headers).json())
            recent2 = recent2[recent2['form'].isin(['10-Q', '10-K'])]
            if (idx % 9 == 0): #There is a time limit in requests. 10 per second
                time.sleep(1)

            dataFrames.append(recent2)

        finalDf = pd.concat(dataFrames).reset_index(drop = True)
        
        finalDf['url'] = 'https://www.sec.gov/Archives/edgar/data/'
        finalDf['url'] += cik + '/' + finalDf['accessionNumber'].str.replace('-', '') + '/' + finalDf['primaryDocument']
        
        finalDf['url_txt'] = 'https://www.sec.gov/Archives/edgar/data/'
        finalDf['url_txt'] += str(int(cik)) + '/' + finalDf['accessionNumber'].str.replace('-', '') + '/' + finalDf['accessionNumber'] + '.txt'
        
        finalDf['filingDate'] = pd.to_datetime(finalDf['filingDate'])
        finalDf['reportDate'] = pd.to_datetime(finalDf['reportDate'])
        
        metadataDictionary[cik] = finalDf
        
    else:
        finalDf = metadataDictionary[cik]
    
    return finalDf
    

In [3]:
df = pd.read_csv('sp500_w_addl_id.csv', parse_dates = [1,11, 12]).iloc[:, 1:]
df['ncusip'] = df['ncusip'].astype(str)
df['date'] = df.date + pd.offsets.MonthEnd(0) 
df['permno'] = df['permno'].astype(np.int64)
print('Data Types')
print(df.dtypes)
print()
print('Shape')
print(df.shape)
df.head(10)

Data Types
date      datetime64[ns]
permno             int64
comnam            object
ncusip            object
shrcd            float64
exchcd           float64
hsiccd           float64
ticker            object
gvkey              int64
iid               object
start     datetime64[ns]
ending    datetime64[ns]
ret              float64
dtype: object

Shape
(66118, 13)


C:\Users\JuliJaramillo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,date,permno,comnam,ncusip,shrcd,exchcd,hsiccd,ticker,gvkey,iid,start,ending,ret
0,2011-01-31,60986,NEWELL RUBBERMAID INC,65122910,11.0,1.0,3089.0,NWL,7875,01,1989-04-27,2022-03-31,0.058856
1,2011-01-31,85914,BEST BUY COMPANY INC,08651610,11.0,1.0,5731.0,BBY,2184,01,1999-06-30,2022-03-31,-0.008457
2,2011-01-31,80711,APARTMENT INVESTMENT & MGMT CO,03748R10,18.0,1.0,6798.0,AIV,30490,01,2003-03-14,2020-12-18,-0.010836
3,2011-01-31,59176,AMERICAN EXPRESS CO,02581610,11.0,1.0,6141.0,AXP,1447,01,1976-07-01,2022-03-31,0.014912
4,2011-01-31,75100,TIFFANY & CO NEW,88654710,11.0,1.0,5944.0,TIF,13646,01,2000-06-21,2021-01-06,-0.066485
5,2011-01-31,76708,DEVRY INC DEL,25189310,11.0,1.0,8221.0,DV,3905,01,2009-06-09,2012-09-28,0.086078
6,2011-01-31,83976,ABERCROMBIE & FITCH CO,00289620,11.0,1.0,5651.0,ANF,63643,01,2007-03-29,2013-12-20,-0.125282
7,2011-01-31,45241,FOREST LABS INC,34583810,11.0,1.0,2834.0,FRX,4843,01,2000-11-22,2014-06-30,0.008755
8,2011-01-31,12067,NOVELLUS SYSTEMS INC,67000810,11.0,3.0,3350.0,NVLS,14623,01,2000-06-19,2012-06-04,0.116027
9,2011-01-31,59408,BANK OF AMERICA CORP,06050510,11.0,1.0,6021.0,BAC,7647,01,1976-07-01,2022-03-31,0.029235


In [4]:
dtype_dic= {'CUSIP':str, 'NCUSIP':str}
returns = pd.read_csv('returns.csv', parse_dates = [1], dtype = dtype_dic)
returns.rename({'PERMNO': 'permno'}, inplace = True, axis = 1)
returns = returns.sort_values(by = 'date')
print(returns.dtypes)
print(returns.head())

permno               int64
date        datetime64[ns]
EXCHCD               int64
NCUSIP              object
TICKER              object
COMNAM              object
PRIMEXCH            object
TRDSTAT             object
PERMCO               int64
CUSIP               object
RCRDDT              object
NWPERM             float64
PRC                float64
VOL                float64
CFACPR             float64
OPENPRC            float64
dtype: object
         permno       date  EXCHCD    NCUSIP TICKER  \
0         10104 2011-01-03       3  68389X10   ORCL   
73198     12076 2011-01-03       1  G8127610    SIG   
1261958   82618 2011-01-03       3  80004C10   SNDK   
1263307   82642 2011-01-03       1  51843910     EL   
1266076   82643 2011-01-03       1  52977110    LXK   

                                COMNAM PRIMEXCH TRDSTAT  PERMCO     CUSIP  \
0                          ORACLE CORP        Q       A    8045  68389X10   
73198              SIGNET JEWELERS LTD        N       A    9857  G812

In [5]:
dtype_dic= {'iid':str, 'cusip':str}
mapping = pd.read_csv('finalMapping.csv', parse_dates = [2], dtype = dtype_dic)
mapping.rename({'datadate':'date'}, axis = 1, inplace = True)
mapping = mapping.sort_values('date')
mapping['cusip'] = mapping['cusip'].str[:8]
print(mapping.dtypes)
mapping.head()

gvkey               int64
iid                object
date       datetime64[ns]
tic                object
cusip              object
conm               object
divd              float64
exchg               int64
secstat            object
cik               float64
dtype: object


,gvkey,iid,date,tic,cusip,conm,divd,exchg,secstat,cik
0,1045,01,2011-01-03,AAMRQ,00176510,AMERICAN AIRLINES GROUP INC,NaN,19,I,6201.0
1549807,29099,05,2011-01-03,REG.PD,75884950,REGENCY CENTERS CORP,NaN,11,I,910606.0
1550121,29099,06,2011-01-03,REG.PE,75884960,REGENCY CENTERS CORP,NaN,11,I,910606.0
121037,2285,01,2011-01-03,BA,09702310,BOEING CO,NaN,11,A,12927.0
1553077,29127,01,2011-01-03,INCY,45337C10,INCYTE CORP,NaN,14,A,879169.0


In [10]:
merged = pd.merge_asof(df, returns, on = 'date', by ='permno', direction = 'backward')
merged.rename({'CUSIP':'cusip'}, inplace = True, axis = 1)
print(merged.isna().sum())
merged.head()

date            0
permno          0
comnam          0
ncusip          0
shrcd           0
exchcd          0
hsiccd          0
ticker          0
gvkey           0
iid             0
start           0
ending          0
ret            34
EXCHCD          0
NCUSIP          0
TICKER          0
COMNAM          0
PRIMEXCH        0
TRDSTAT         0
PERMCO          0
cusip           0
RCRDDT      65047
NWPERM      66100
PRC             0
VOL             0
CFACPR          0
OPENPRC         0
dtype: int64


,date,permno,comnam,ncusip,shrcd,exchcd,hsiccd,ticker,gvkey,iid,...,PRIMEXCH,TRDSTAT,PERMCO,cusip,RCRDDT,NWPERM,PRC,VOL,CFACPR,OPENPRC
0,2011-01-31,60986,NEWELL RUBBERMAID INC,65122910,11.0,1.0,3089.0,NWL,7875,01,...,N,A,3178,65122910,NaN,NaN,19.25,3631600.0,1.000000,19.43
1,2011-01-31,85914,BEST BUY COMPANY INC,08651610,11.0,1.0,5731.0,BBY,2184,01,...,N,A,7506,08651610,NaN,NaN,34.00,6034700.0,1.000000,34.01
2,2011-01-31,80711,APARTMENT INVESTMENT & MGMT CO,03748R10,18.0,1.0,6798.0,AIV,30490,01,...,N,A,30438,03748R74,NaN,NaN,25.56,1627300.0,8.766256,25.08
3,2011-01-31,59176,AMERICAN EXPRESS CO,02581610,11.0,1.0,6141.0,AXP,1447,01,...,N,A,90,02581610,NaN,NaN,43.38,9931500.0,1.000000,44.13
4,2011-01-31,75100,TIFFANY & CO NEW,88654710,11.0,1.0,5944.0,TIF,13646,01,...,N,A,21753,88654710,NaN,NaN,58.13,966300.0,1.000000,57.80


In [ ]:
final = pd.merge_asof(merged, mapping, on = 'date', by ='gvkey', direction = 'nearest')
dict_ = {'APARTMENT INVESTMENT & MGMT CO': 922864,
 'ALCOA INC': 1675149,
 'PEABODY ENERGY CORP': 1064728,
 'CHESAPEAKE ENERGY CORP': 895126,
 'JACOBS ENGINEERING GROUP INC': 52988,
 'JANUS CAP GROUP INC': 1065865,
 'METROPCS COMMUNICATIONS INC': 1121026,
 'WISCONSIN ENERGY CORP': 783325,
 'RADIOSHACK CORP': 81727,
 'OWENS ILL INC': 861326,
 'NEWS CORP': 1564708,
 'WINDSTREAM CORP': 1282266,
 'WYNDHAM WORLDWIDE CORP': 1010471,
 'J D S UNIPHASE CORP': 1257179,
 'CONSOL ENERGY INC': 1710366,
 'YAHOO INC': 1426140,
 'DEAN FOODS CO': 931336,
 'DONNELLEY R R & SONS CO': 29669,
 'FRONTIER COMMUNICATIONS CORP': 20520,
 'SARA LEE CORP': 1080839,
 'I T T CORP': 216228,
 'NABORS INDUSTRIES LTD': 1163739,
 'SUPERVALU INC': 95521,
 'TWENTY FIRST CENTURY FOX INC': 1308161,
 'WINDSTREAM HOLDINGS INC': 1282266,
 'S L GREEN REALTY CORP': 1040971,
 'W E C ENERGY GROUP INC': 783325,
 'FEDERAL REALTY INVESTMENT TRUST': 34903,
 'G G P INC': 895648,
 'DISCOVERY INC': 1437107,
 'HOLLYFRONTIER CORP': 48039, 'CLIFFS NATURAL RESOURCES INC': np.nan, 'DEVRY INC DEL':np.nan, 
         'M E M C ELECTRONIC MATERIALS INC':np.nan, 'COCA COLA ENTERPRISES INC NEW': np.nan, 
         'DISCOVERY COMMUNICATIONS INC':np.nan, 'DR PEPPER SNAPPLE GROUP INC': np.nan, 'DEAN FOODS CO NEW': np.nan,
         'PENNEY J C CO INC':np.nan, 'E N S C O PLC NEW':np.nan, 'GENERAL GROWTH PPTYS INC NEW': np.nan
    
        }

temp = final[final.cik.isna()]
temp['cik'] = temp.apply(lambda row: dict_[row['comnam']] if np.isnan(row['cik']) else row['cik'], axis=1)
temp[['comnam', 'cik']]
final[final.cik.isna()] = temp
final = final.dropna(subset=['cik'])
del temp

print(final.shape)
final.isna().sum()

In [ ]:
uniqueCIKs = final.dropna(subset = ['cik']).cik.unique()
urlList = []
for cik in uniqueCIKs:
    try:
        cik_search = str(int(cik)).zfill(10)

        meta = metadata(cik_search)
        meta['reportDate2'] = meta['reportDate'].dt.strftime('%Y-%m')

        tempDf = final[final['cik'] == cik]
        tempDf['date2'] = tempDf['date'].dt.strftime('%Y-%m')

        mergedTempDf = pd.merge(tempDf, meta, left_on = 'date2', right_on = 'reportDate2')
        urlList.append(mergedTempDf)
    except:
        print('{} not found'.format(cik))
    
finalUrls = pd.concat(urlList)
finalUrls.head()

In [ ]:
'''
#Write the unique ncusips to a txt file
uniqueGVKEY = df['gvkey'].unique()
with open('gvkey.txt', 'w') as f:
    for gvkey in uniqueGVKEY:
        f.write(str(gvkey))
        f.write('\n')
'''

In [ ]:
'''
mapping = pd.read_csv('mapping.csv', parse_dates = ['LINKDT', 'LINKENDDT'])
mapping.loc[mapping['LINKENDDT'] == 'E', 'LINKENDDT'] = '20220909'
mapping['LINKENDDT'] = pd.to_datetime( mapping['LINKENDDT'] )
mapping['cusip'] = mapping['cusip'].str[0:8]
#mapping = pd.read_csv('mapping2.csv', parse_dates = ['datadate'])
mapping = mapping.drop_duplicates(subset = ['gvkey', 'cusip', 'cik'])
print(mapping.dtypes)
print(mapping.shape)
print(mapping.nunique())
mapping.head()

merged = pd.merge(df, mapping, left_on = 'gvkey', right_on = 'gvkey', how = 'left')
#merged = merged.loc[(merged['date'] >= merged['LINKDT']) & (merged['date'] <= merged['LINKENDDT'])]
print(merged.isna().sum())
print('df shape', df.shape)
print('mapping shape', mapping.shape)
print('merged shape', merged.shape)
merged.head(10)

#Search which are the gvkeys for which the CIK was nan
NaNgvkey =  merged[merged['cik'].isna()].gvkey.unique()
print(NaNgvkey)

merged.loc[merged.gvkey == 1356, 'cik'] = 1675149
merged.loc[merged.gvkey == 137232 , 'cik'] = 1065865
merged.loc[merged.gvkey == 160256, 'cik'] = 1121026
merged.loc[merged.gvkey == 13353, 'cik'] = 783325
merged.loc[(merged.gvkey == 4199) & (merged.comnam == 'EATON CORP'), 'cik'] = 31277
merged.loc[(merged.gvkey == 4199) & (merged.comnam == 'EATON CORP PLC'), 'cik'] = 1551182
merged.loc[merged.gvkey == 179700, 'cik'] = 1418135
merged.loc[merged.gvkey == 23546, 'cik'] = 1725057

finalDf = merged[['date', 'comnam', 'cik', 'gvkey', 'cusip', 'ncusip']]
finalDf['cik'] = finalDf['cik'].astype(int).astype(str)
finalDf['cik_10'] = finalDf['cik'].str.pad(10, fillchar = '0')

print(finalDf.dtypes)
print(finalDf.head())
print(finalDf.tail())
finalDf[finalDf['date'] == '2011-01-31']

dates = pd.DataFrame( pd.date_range(start = "2010-12-31", end = "2021-12-31", freq = "Q"), columns = ['date_Q'] )
dates.head()

finalDfQ = pd.merge(left = dates.iloc[1:,:], right = finalDf, left_on = 'date_Q', right_on = 'date', how = 'left')
print(finalDfQ.isna().sum())
print(finalDfQ.dtypes)
print(finalDfQ.head())
print(finalDfQ.tail())

theDates = pd.date_range(start = "2011-01-01", end = "2022-03-31", freq = "Q")
dictionary = {}
anomalies = []
for idx, date in tqdm(enumerate(theDates[:-1])):
    print(date)
    
    currentDate = date
    nextDate = theDates[idx+1]
    tempDf = finalDfQ[finalDfQ['date'] == date]
    
    for idx, cik in enumerate(tempDf['cik']):
        cik_10 = cik.zfill(10)
        metadataDf = metadata(cik_10)
        metadataDf = metadataDf[(metadataDf['filingDate'] >= currentDate) & (metadataDf['filingDate'] <= nextDate)]
        #metadataDf = metadataDf[(metadataDf['reportDate'] >= currentDate) & (metadataDf['reportDate'] <= nextDate)]
        if (metadataDf.shape[0] == 1):
            keyDate = str(currentDate).split(' ')[0]
            tuple_to_insert = (metadataDf.iloc[0, -2], metadataDf.iloc[0, -1], metadataDf.iloc[0, 1], metadataDf.iloc[0, 2])
            if keyDate not in dictionary:
                dictionary[keyDate] = [ tuple_to_insert ]
            else:
                dictionary[keyDate].append( tuple_to_insert )
        else:
            anomal = (date, cik, metadataDf.shape[0])
            anomalies.append(anomal)
            #print(anomal)
                
        if (idx % 200 == 0):
            print(cik)
            
anomalies = pd.DataFrame(anomalies, columns = ['date', 'cik', 'rows'])

    
pickle.dump( dictionary, open( "urls.pkl", "wb" ) )
pickle.dump( metadataDictionary, open( "metadata.pkl", "wb" ) )

anomalies['cik'] = anomalies['cik'].astype(str)
anomalies = pd.merge(anomalies, finalDfQ, left_on = ['date', 'cik'], right_on = ['date', 'cik'], how = 'inner')
anomalies.to_csv('anomalies.csv')

finalDfQ.to_csv('finalDfQ.csv')

a = metadataDictionary['0001288784']
#a[(a['filingDate'] >= currentDate) & (a['filingDate'] <= nextDate)]
a

'''

In [ ]:
finalUrls.to_csv('finalUrls.csv', index = False)